In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import rigging as rg
import mlframework.arc.images as images
from mlframework.files.json import load_json
import arckit
import pathlib

In [ ]:
PATH_DATA = pathlib.Path("/home/kristian/Projects/mlframework/data/")
PATH_COMPETITION = PATH_DATA / "arc/arc-prize-2024/"
PATH_TRAIN_CHALLENGES = PATH_COMPETITION / "arc-agi_training_challenges.json"
PATH_TRAIN_SOLUTIONS = PATH_COMPETITION / "arc-agi_training_solutions.json"
PATH_TEST = PATH_COMPETITION / "test.csv"
PATH_SUBMISSION_EXAMPLE = PATH_COMPETITION / "sample_submission.csv"

# MODEL = "transformers!meta-llama/Meta-Llama-3-8B-Instruct,device_map=cuda:1,max_tokens=1024,load_in_4bit=True"
MODEL = "openai/gpt-4o-mini"

In [ ]:
train_solutions = load_json(PATH_TRAIN_SOLUTIONS)

train_set, test_set = arckit.load_data("kaggle2024")

task_example = train_set.tasks[0]
drawing = images.show_task(task_example, train_solutions=train_solutions)

In [ ]:
prompt = task_example.gpt_prompt(0, include_completion=False)
prompt

In [ ]:
def array_to_str(array):
    return array.__str__().strip("[]").replace("]", "\n").replace("[", "").replace(",", "")
def sample_to_str(sample):
    examples_dict = sample.to_dict()
    training_examples_str = ""
    for i, example in enumerate(examples_dict["train"]):
        for io in ["input", "output"]:
            training_examples_str += f"{io} {i}: " + array_to_str(example[io]) + "\n"
    return training_examples_str

In [ ]:
class Solution(rg.Model):
    solution: str
    explanation: str

def get_model():
    return rg.get_generator(MODEL)

async def generate_solution(sample, model, verbose=False):
    system_prompt = """
        'We are playing a game which involves transforming an input grid of digits into an output grid of digits. 
        In general, digits form objects in 2D and the task is to perform some spatial transformation 
        of these objects to go from the input grid to the output grid. 
        All the information about the transformation is contained within the input pairs themselves, 
        and your answer will only be correct if the output grid is exactly correct, 
        so this is what I expect from you. I will begin by giving you several examples of input-output pairs. 
        You will then be given a new input grid, and you must provide the corresponding output grid.\n
        
        Please provide a step-by-step explanation. 
        Specifically, answer the following in your explanation. 
        1. Justify the output shape of your answer. \n
        2. Did you consider shapes in the outputs and why?\n
        Provide the output for Input 6 again at the end of your answer.'
    """
    sample_str = sample_to_str(sample)

    user_prompt = f"""
        The training sample is {sample_str}
    
        Provide your solution and explanation within the tag {Solution.xml_tags()}.
    """
    asker = (
        await model
        .chat(
            [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ]
        )
        .run()
    )
    solution = asker.last.parse(Solution).solution
    explanation = asker.last.parse(Solution).explanation
    if verbose:
        print(f"=== Solution ====")
        print(solution)
        print(f"=== Explanation ====")
        print(explanation)

    return solution, explanation

In [ ]:
model = get_model()

In [ ]:
await generate_solution(sample=task_example, model=model, verbose=True)